In [ ]:
!pip install obonet pandas numpy tensorflow scikit-learn
!pip install obonet pandas numpy tensorflow scikit-learn
import pandas as pd
import numpy as np
import obonet
import networkx as nx
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import binary_crossentropy


In [ ]:
# Chemins d'accès aux fichiers (basés sur votre input)
EMBEDDINGS_PATH = '/kaggle/input/test-protein-embedding-complet/test_protein_embeddings_complet.npy'
IDS_PATH = '/kaggle/input/test-protein-embedding-complet/test_protein_ids_complet.npy'
TERMS_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
OBO_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'

# --- Chargement des embeddings (X) et des IDs ---
X = np.load(EMBEDDINGS_PATH)
train_ids = np.load(IDS_PATH)
EMBEDDING_DIM = X.shape[1]
print(f"Embeddings chargés. Dimension X: {X.shape}")

# --- Chargement des Labels (Y) ---
train_terms = pd.read_csv(TERMS_PATH, sep='\t')
print(f"Termes chargés. Nombre de lignes: {len(train_terms)}")

train_terms = pd.read_csv(TERMS_PATH, sep='\t')
print("Noms exacts des colonnes dans train_terms.tsv:")
print(train_terms.columns.tolist())

In [ ]:
# 1. Sélection des termes GO (les labels)
# Nous utilisons seulement les termes GO présents dans le jeu d'entraînement.
# Nous allons filtrer les termes trop rares pour l'entraînement (par exemple, si moins de 50 protéines ont ce terme).
TERM_COUNT_THRESHOLD = 50
term_counts = train_terms['term'].value_counts()
selected_terms = term_counts[term_counts >= TERM_COUNT_THRESHOLD].index.tolist()

# 2. Création du mappage ID -> Index
protein_to_index = {pid: i for i, pid in enumerate(train_ids)}
term_to_index = {term: i for i, term in enumerate(selected_terms)}
num_classes = len(selected_terms)

# 3. Construction de la matrice Y binaire
Y_matrix = np.zeros((len(train_ids), num_classes), dtype=np.int8)

# Remplissage de la matrice Y
for _, row in train_terms.iterrows():
    protein_id = row['EntryID']
    term = row['term']
    
    if protein_id in protein_to_index and term in term_to_index:
        p_idx = protein_to_index[protein_id]
        t_idx = term_to_index[term]
        Y_matrix[p_idx, t_idx] = 1

print(f"Matrice de labels Y créée. Dimension Y: {Y_matrix.shape}")

In [ ]:
# --- Chargement du graphe GO et calcul des Poids d'IA ---
print("\n--- 3. Calcul de l'Information Content (IA) ---")

# Chargement du graphe (nécessite obonet)
try:
    go_graph = obonet.read_obo(OBO_PATH)
except Exception as e:
    print(f"Erreur lors du chargement d'obonet. Assurez-vous que 'obonet' est installé. Erreur: {e}")
    # Si le chargement échoue, on continue avec un IC basé uniquement sur la fréquence
    # sans utiliser la hiérarchie GO pour le moment (moins précis mais fonctionnel)
    pass


# Calcul des fréquences d'apparition (pour l'Information Content - IC)
# On compte combien de protéines ont chaque terme dans le jeu Y_matrix
term_frequencies = Y_matrix.sum(axis=0)

# Calcul de l'Information Content (IC)
# IC = -log2(P(t)) où P(t) est la probabilité d'un terme t
total_proteins = len(train_ids)
term_probabilities = term_frequencies / total_proteins
# Le terme le plus fréquent (P proche de 1) aura un IC proche de 0
# Le terme le plus rare (P proche de 0) aura un IC très élevé
# On ajoute une petite valeur (epsilon) pour éviter log(0)
epsilon = 1e-10
IC_values = -np.log2(term_probabilities + epsilon)

# Normalisation pour la fonction de perte
# Plus l'IC est élevé, plus le poids sera grand.
# On normalise pour que le poids maximum soit 1.0 (ou proche).
class_weights = IC_values / IC_values.max() 
class_weights_tensor = tf.constant(class_weights, dtype=tf.float32)

print(f"Poids d'Information Content (IA) calculés. Nombre de classes: {len(class_weights)}. Max weight: {class_weights.max():.2f}")

In [ ]:
# Inspecter le poids
print("Valeurs des poids (IA) :")
print(class_weights_tensor)
print(f"Nombre de poids nuls : {tf.reduce_sum(tf.cast(class_weights_tensor == 0, tf.int32))}")

In [ ]:
# --- Étape A : Inspection des IDs ---
# Récupérer les 5 premiers IDs de la source label (train_terms.tsv)
# Utilisez le nom de colonne 'EntryID' original, puis le nettoyage.
ids_terms_original = train_terms['EntryID'].head().tolist()
ids_terms_nettoyes = train_terms['EntryID'].str.split('.').str[0].head().tolist()

# Récupérer les 5 premiers IDs de la source embeddings (train_ids.npy)
ids_embeddings = train_ids[:5].tolist()


print("IDs labels (EntryID original) :", ids_terms_original)
print("IDs labels (Après .split('.')) :", ids_terms_nettoyes)
print("IDs embeddings (train_ids.npy) :", ids_embeddings)

In [ ]:
# --- Étape de CORRECTION DÉFINITIVE (Ré-exécutez tout le bloc de nettoyage) ---

# 1. Nettoyage des IDs dans les labels (train_terms.tsv)
# Bien que les exemples soient déjà propres, on garde le split pour la sécurité
train_terms['EntryID_CLEAN'] = train_terms['EntryID'].str.split('.').str[0]

# 2. Nettoyage CRITIQUE des IDs d'embeddings (train_ids.npy)
# On extrait l'ID Uniprot brut (le segment entre les barres verticales '|')
def extract_uniprot_id(full_id):
    """Extrait l'ID Uniprot brut de la chaîne FASTA complète."""
    try:
        # La 2e position (index 1) après le split par '|' est l'ID Uniprot
        return full_id.split('|')[1]
    except IndexError:
        # Retourne l'ID original si le format n'est pas trouvé (sécurité)
        return full_id

# Appliquer la fonction pour créer les IDs propres des embeddings
train_ids_cleaned = np.array([extract_uniprot_id(str(i)) for i in train_ids]) 

# 3. Recréation du Dictionnaire de Mapping (avec les IDs nettoyés)
# Les clés seront maintenant les IDs bruts (ex: 'A0A0C5B5G6')
protein_to_index = {pid: i for i, pid in enumerate(train_ids_cleaned)}

# Vérification pour s'assurer qu'au moins quelques IDs correspondent (optionnel mais utile)
print(f"\nExemples d'IDs nettoyés des embeddings : {train_ids_cleaned[:5].tolist()}")
print(f"L'ID 'Q5W0B1' est-il dans le dictionnaire ? {'Q5W0B1' in protein_to_index}")


# --- 4. Remplissage de la matrice Y (CODE FINAL) ---
# Re-initialiser la matrice Y 
Y_matrix = np.zeros((len(train_ids_cleaned), num_classes), dtype=np.int8)

filled_count = 0
for _, row in train_terms.iterrows():
    # Utiliser l'ID nettoyé du terme
    protein_id_clean = str(row['EntryID_CLEAN'])
    term = row['term']
    
    # La condition devrait maintenant être VRAIE si l'ID existe dans le jeu d'embeddings
    if protein_id_clean in protein_to_index and term in term_to_index:
        p_idx = protein_to_index[protein_id_clean]
        t_idx = term_to_index[term]
        Y_matrix[p_idx, t_idx] = 1
        filled_count += 1

# --- VÉRIFICATION FINALE ---
print(f"\nNombre total de labels positifs ('1') remplis : {Y_matrix.sum()}")
print(f"Nombre total de lignes traitées : {filled_count}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# --- DÉFINITION DE L'ARCHITECTURE MLP ) ---
# Ceci crée la variable 'model'
model = Sequential([
    # Couche d'entrée : la dimension est la taille des embeddings (320)
    Dense(1024, activation='relu', input_shape=(320,)), 
    Dropout(0.2),
    # Couche cachée intermédiaire (peut varier)
    Dense(512, activation='relu'),
    Dropout(0.2),
    # Couche de sortie : 1585 neurones (le nombre de termes GO) avec activation sigmoïde pour la multi-classification
    Dense(1585, activation='sigmoid') 
])

# Vérifiez l'architecture
model.summary()


In [ ]:
# --- Définition de la Perte, de l'Architecture et Entraînement ---

# 1. Définir la fonction de perte pondérée (si les poids IA sont déjà calculés et variés)
# Nous avions une perte de validation très faible précédemment. Nous allons
# nous assurer que le modèle apprend quelque chose en utilisant la perte standard.

# Si vous voulez avancer sans risque de bug de la perte IA:
# loss_function = 'binary_crossentropy' 

# Si vous voulez essayer à nouveau la perte IA corrigée (en espérant que le bug était lié à Y nul):
# loss_function = weighted_binary_crossentropy 


# --- Séparation des données pour l'entraînement et la validation ---
X_train, X_val, Y_train, Y_val = train_test_split(
    X, 
    Y_matrix, 
    test_size=0.1, 
    random_state=42 
)

# --- Compilation et Entraînement (avec binary_crossentropy pour la sécurité) ---
# Si le GPU est détecté, l'entraînement sera rapide.
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', # UTILISATION DE LA PERTE STANDARD POUR VÉRIFICATION
    metrics=[tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

print("\nDébut de l'entraînement du modèle MLP...")

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True
)

history = model.fit(
    X_train, Y_train,
    epochs=50, 
    batch_size=64,
    validation_data=(X_val, Y_val),
    callbacks=[early_stopping] 
)

print("\nEntraînement terminé.")

In [ ]:
# second entraînement
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np # Assurez-vous d'avoir importé numpy si class_weights_tensor vient d'un numpy array

# --- Fonction de perte pondérée par l'IA (CORRIGÉE) ---
def weighted_binary_crossentropy_FINAL(y_true, y_pred):
    """
    Fonction de perte pondérée par l'Information Content (IA).
    Utilise le backend de Keras pour gérer correctement les formes.
    """
    # 1. Calcul de la BCE pour chaque élément (sans réduction)
    # y_true et y_pred sont de forme (batch_size, 1585)
    bce = K.binary_crossentropy(y_true, y_pred) 
    
    # 2. Ajout de dimension (1585,) -> (1, 1585) pour le broadcasting sécurisé
    # Ceci assure que chaque ligne du batch est multipliée par le vecteur de poids.
    weights = class_weights_tensor[None, :] 
    
    # 3. Multiplication élément par élément : (64, 1585) * (1, 1585)
    weighted_bce = bce * weights
    
    # 4. Retourne la moyenne de la perte sur toutes les classes et le batch
    return K.mean(weighted_bce)

# --- Recréer le Tenseur de Poids si nécessaire ---
# Si votre class_weights_tensor a été perdu ou redéfini, assurez-vous qu'il soit bien un tenseur TF
# class_weights_tensor = tf.constant(votre_array_de_poids_IA_de_numpy, dtype=tf.float32)

print("Fonction de perte pondérée par l'IA mise à jour et chargée.")

In [ ]:
# --- Compilation du Modèle avec la Perte IA ---

print("\n--- Relance de la compilation avec la Perte IA ---")

# Recompilez le modèle en utilisant la nouvelle fonction de perte
model.compile(
    optimizer='adam',
    # UTILISATION DE LA PERTE PONDÉRÉE PAR L'IA CORRIGÉE
    loss=weighted_binary_crossentropy_FINAL, 
    metrics=[tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

# --- Entraînement ---

print("\nDébut du second entraînement (Optimisation Fmax)...")

# Lancer le nouvel entraînement
# L'EarlyStopping utilisera la 'val_loss' pondérée pour déterminer le meilleur point d'arrêt
history_ia = model.fit(
    X_train, Y_train,
    epochs=50, # Vous pouvez augmenter les époques si le premier entrainement a utilisé moins de 20
    batch_size=64,
    validation_data=(X_val, Y_val),
    callbacks=[early_stopping] 
)

print("\nEntraînement avec la Perte IA terminé.")
NEW_WEIGHTS_FILE = 'best_cafa6_mlp_ic_weighted.weights.h5' 
model.save_weights(NEW_WEIGHTS_FILE)

print(f"Sauvegarde des poids terminée : {NEW_WEIGHTS_FILE}")

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

# --- 0. Définition des chemins des sources de la compétition ---
# Ces fichiers sont nécessaires pour la recréation
TRAIN_TERMS_FILE = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'

# --- CORRECTION CLEF : RECRÉATION DU MAPPING GO À PARTIR DE LA SOURCE ---
print("Tentative de recréation du mapping GO...")
try:
    # 1. Charger le fichier de termes source
    df_terms = pd.read_csv(TRAIN_TERMS_FILE, sep='\t')
    
    # 2. Identifier les 1585 termes que vous avez utilisés (les plus fréquents)
    # On suppose que vous avez filtré sur les termes les plus fréquents (> 220 occurrences, le seuil standard)
    term_counts = df_terms['term'].value_counts()
    
    # Définir le seuil utilisé pour la compétition (typiquement 220 pour avoir ~1585 termes)
    # Ceci doit correspondre au seuil que vous avez utilisé lors de la création de Y !
    # Si le nombre de termes est 1585, le seuil était probablement autour de 220.
    TARGET_TERM_COUNT = 1585 
    
    # Ajustement heuristique pour obtenir le bon nombre de termes (cela suppose un ordre précis)
    # Pour l'inférence, nous allons simplement prendre les 1585 termes les plus fréquents.
    # C'est la méthode la plus probable que vous ayez utilisée.
    
    # Prendre les termes les plus fréquents dans l'ordre de leur fréquence (hypothèse la plus forte)
    go_terms_list_recreated = term_counts.head(TARGET_TERM_COUNT).index.tolist()
    
    # Créer le dictionnaire de conversion (index -> ID GO)
    index_to_term = {i: term for i, term in enumerate(go_terms_list_recreated)}
    
    print(f"✅ Mapping index -> GO ID recréé avec succès. Total termes : {len(index_to_term)}")
    
    # --- Continuer le chargement des autres fichiers pour l'inférence ---
    
    # ... (code de chargement X_test et test_ids_clean) ...
    
    # Ensuite, le code de prédiction et de soumission...
    
except Exception as e:
    print(f"❌ ÉCHEC : Impossible de recréer le mapping GO à partir de {TRAIN_TERMS_FILE}. Détails : {e}")
    index_to_term = {}

In [ ]:
import numpy as np
import tensorflow as tf
import os 
import pandas as pd
from tensorflow.keras import backend as K

# --- 0. Initialisation et Définitions ---

# Chemins d'accès confirmés par l'utilisateur (utilisés pour l'inférence)
TEST_IDS_FILE = '/kaggle/input/test-protein-embedding-complet/test_protein_ids_complet.npy'
TEST_EMBEDDINGS_FILE = '/kaggle/input/test-protein-embedding-complet/test_protein_embeddings_complet.npy' 
TRAIN_TERMS_FILE = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'

# Assurez-vous que 'model' (avec les poids entraînés) et 'class_weights_tensor' sont définis dans votre notebook !

def weighted_binary_crossentropy_FINAL(y_true, y_pred):
    # La fonction de perte doit être définie si le modèle est recompilé
    bce = K.binary_crossentropy(y_true, y_pred) 
    weights = class_weights_tensor[None, :] 
    weighted_bce = bce * weights
    return K.mean(weighted_bce)

try:
    # Re-compiler le modèle pour s'assurer que les métriques sont disponibles (les poids sont restaurés par EarlyStopping)
    model.compile(
        optimizer='adam',
        loss=weighted_binary_crossentropy_FINAL, 
        metrics=[tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
    )
except NameError:
    print("❌ ATTENTION : La variable 'model' ou 'class_weights_tensor' n'est pas définie. Veuillez recharger le modèle.")

# --- 1. CHARGEMENT DES DONNÉES DE TEST ET RECRÉATION DU MAPPING GO ---
print("\n--- 1. Chargement et Recréation du Mapping ---")

# 1. Charger les IDs de test
try:
    test_ids = np.load(TEST_IDS_FILE) 
    test_ids_clean = np.array([i.split('|')[1] if '|' in str(i) else str(i) for i in test_ids])
    print(f"✅ IDs chargés. Nombre de protéines : {len(test_ids_clean)}")
except Exception as e:
    print(f"❌ Erreur de chargement IDs : {e}")
    test_ids_clean = np.array([]) 

# 2. Charger les embeddings de test (X_test)
try:
    X_test = np.load(TEST_EMBEDDINGS_FILE) 
    print(f"✅ Embeddings chargés. Forme : {X_test.shape}")
except Exception as e:
    print(f"❌ Erreur de chargement Embeddings : {e}")
    X_test = np.array([])

# 3. RECRÉATION du mapping des termes GO (Solution de déblocage)
try:
    df_terms = pd.read_csv(TRAIN_TERMS_FILE, sep='\t')
    term_counts = df_terms['term'].value_counts()
    # On suppose que 1585 termes correspondent aux 1585 plus fréquents
    go_terms_list_recreated = term_counts.head(1585).index.tolist()
    index_to_term = {i: term for i, term in enumerate(go_terms_list_recreated)}
    print(f"✅ Mapping index -> GO ID recréé. Total termes : {len(index_to_term)}")
except Exception as e:
    print(f"❌ ÉCHEC FATAL : Impossible de recréer le mapping GO. {e}")
    index_to_term = {} 



In [ ]:
#  INFÉRENCE (CRITIQUE) ---

print("Début de l'inférence sur le jeu de test complet...")

# ⚠️ X_test doit contenir les embeddings complets de test (forme: 224309, 320)
# La variable Y_pred_proba stockera les probabilités de prédiction brutes du MLP
Y_pred_proba = model.predict(X_test) 

print(f"✅ Inférence terminée. Dimensions des prédictions brutes : {Y_pred_proba.shape}")
print(f"Le modèle a prédit {Y_pred_proba.shape[0]} protéines sur {Y_pred_proba.shape[1]} termes GO.")


In [ ]:
# Propagation GO
!pip install obonet networkx

import networkx as nx
import numpy as np

import obonet
# Remplacez 'chemin/vers/go-basic.obo' par le chemin réel dans votre Kaggle Input
OBO_FILE_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo' 
go_graph = obonet.read_obo(OBO_FILE_PATH)
print(f"Graphe GO chargé avec {go_graph.number_of_nodes()} nœuds.")


# --- 1. SÉCURITÉ ET PRÉPARATION ---

# Crée une copie des prédictions brutes pour les modifier (très important !)
Y_pred_proba_propagated = np.copy(Y_pred_proba)

# Récupère l'ensemble des termes GO prédits par le modèle
go_terms_in_model = set(index_to_term.values())

# --- 2. FONCTION DE PROPAGATION HIERARCHIQUE ---

# Nous allons itérer sur les nœuds du graphe GO par ordre topologique inverse
# (des feuilles vers les racines) pour assurer que les parents sont mis à jour après les enfants.

# On utilise les termes qui sont à la fois dans notre modèle et dans le graphe GO
nodes_to_propagate = [n for n in nx.topological_sort(go_graph) if n in go_terms_in_model]
# Inverser l'ordre : du bas (feuilles) vers le haut (racines)
nodes_to_propagate.reverse()

print(f"\nDébut de la propagation pour {len(nodes_to_propagate)} termes GO...")

# --- 3. APPLICATION DE LA PROPAGATION ---

for term_id_child in nodes_to_propagate:
    try:
        # Récupère l'index de ce terme dans la matrice de prédiction
        idx_child = term_to_index[term_id_child]

        # Trouvez tous les parents directs (relations 'is a' ou 'part of')
        # On utilise go_graph.predecessors(node) pour trouver les parents directs
        # dans le graphe OBO où les arêtes pointent d'enfant à parent.
        for term_id_parent in go_graph.predecessors(term_id_child):
            
            # Assurez-vous que le parent est un terme que notre modèle prédit (un des 1585)
            if term_id_parent in term_to_index:
                idx_parent = term_to_index[term_id_parent]
                
                # Règle de propagation: Score(Parent) = max(Score(Parent), Score(Enfant))
                # np.maximum effectue cette opération ligne par ligne (pour toutes les protéines à la fois)
                
                # Mise à jour des scores du parent pour toutes les protéines
                Y_pred_proba_propagated[:, idx_parent] = np.maximum(
                    Y_pred_proba_propagated[:, idx_parent], 
                    Y_pred_proba_propagated[:, idx_child]
                )

    except KeyError:
        # Ignore les termes qui sont dans le graphe GO mais que nous n'avons pas sélectionnés (pas l'un des 1585)
        continue 

print("✅ Propagation GO terminée.")
print(f"Les prédictions finales sont stockées dans Y_pred_proba_propagated (Forme: {Y_pred_proba_propagated.shape}).")



In [ ]:
import numpy as np
import os
from operator import itemgetter

# --- 3. FORMATAGE ET ÉCRITURE DU FICHIER DE SOUMISSION CONFORME CAFA ---
print("\n--- 3. Formatage et Écriture du Fichier de Soumission CAFA FINAL ---")

# ⚠️ L'ENTRÉE EST MAINTENANT LE RESULTAT DE LA PROPAGATION GO
Y_FINAL_SCORES = Y_pred_proba_propagated

# 🚨 Rappel de la contrainte (score minimum pour ce type de compétition)
# Le score DOIT être dans l'intervalle (0, 1.000]. Votre seuil est 0.01.
# Le minimum requis est de 0.5 (selon vos informations mémorisées).
# Cependant, pour le formatage CAFA, on garde souvent un seuil bas (0.01) 
# pour inclure tous les termes avant la limite de 1500.

if len(Y_FINAL_SCORES) > 0 and len(index_to_term) == 1585:
    
    # Le score DOIT être dans l'intervalle (0, 1.000].
    SEUIL_SOUMISSION = 0.01 # Gardons ce seuil pour la capture initiale
    # La limite est de 1500 termes MAX par protéine.
    MAX_TERMS_PER_PROTEIN = 1500
    
    submission_lines = []

    for i in range(len(test_ids_clean)):
        protein_id = test_ids_clean[i]
        # Utilisez les scores finaux propagés
        probabilities = Y_FINAL_SCORES[i]
        
        predictions = []
        
        # 1. Identifier les prédictions au-dessus du seuil (doit être > 0.0)
        indices_predits = np.where(probabilities > SEUIL_SOUMISSION)[0]
        
        # Créer des paires (terme, probabilité) pour le tri
        for idx in indices_predits:
            term_id = index_to_term.get(idx)
            prob = probabilities[idx]
            
            if term_id and term_id != 'GO:UNKNOWN':
                predictions.append((term_id, prob))

        # 2. Trier par probabilité (du plus haut au plus bas)
        predictions.sort(key=itemgetter(1), reverse=True)
        
        # 3. Appliquer la limite de 1500 termes
        final_predictions = predictions[:MAX_TERMS_PER_PROTEIN]
        
        # 4. Formatage des lignes
        for term_id, prob in final_predictions:
            # Score doit être dans (0, 1.000] avec max 3 chiffres significatifs.
            formatted_prob = f"{prob:.3f}"
            
            # Format: ID_prot [TAB] GO_ID [TAB] Score (une association par ligne)
            submission_lines.append(f"{protein_id}\t{term_id}\t{formatted_prob}")

    # --- ÉCRIRE LE FICHIER (SANS HEADER) ---
    
    # 💥 CORRECTION CRITIQUE : Le nom du fichier DOIT être 'submission.tsv' pour Kaggle.
    SUBMISSION_FILE_NAME = 'submission.tsv'

    with open(os.path.join('/kaggle/working', SUBMISSION_FILE_NAME), 'w') as f:
        # CRITIQUE : Écriture des lignes SANS HEADER
        f.write('\n'.join(submission_lines))

    print(f"\n✅ Fichier de soumission créé : {SUBMISSION_FILE_NAME}. Nombre total de prédictions : {len(submission_lines)}")
    print("Le fichier est maintenant nommé correctement pour la soumission à Kaggle.")
else:
    print("\n❌ ÉCHEC FINAL : Vérifiez le chargement de Y_FINAL_SCORES (Y_pred_proba_propagated) ou la taille de index_to_term.")